In [1]:
import sqlite3
from PIL import Image
from io import BytesIO
import base64
from tqdm import tqdm

In [2]:
conn = sqlite3.connect("scryfall.db", check_same_thread=False)
cur = conn.cursor()

new_conn = sqlite3.connect("scryfall_new.db", check_same_thread=False)
new_cur = new_conn.cursor()

cmd = """
CREATE TABLE IF NOT EXISTS cards (
    name TEXT PRIMARY KEY,
    mana_cost TEXT,
    cmc INTEGER,
    type_line TEXT,
    oracle_text TEXT,
    power INTEGER,
    toughness INTEGER,
    loyalty INTEGER,
    img_file_name TEXT
);
"""

new_cur.execute(cmd)
new_conn.commit()

In [3]:
not_allowed = [" ", "\\", "/", ":", "*", "?", '"', "<", ">", "|", "'", "(", ")", ",", "!", "@", "#", "$", "%", "^", "&", "`", "~"]

def card_name_to_filename(name):
    file_path: str = name
    for char in not_allowed:
        file_path = file_path.replace(char, "-")
    while "--" in file_path:
        file_path = file_path.replace("--", "-")
    if file_path.endswith("-"):
        file_path = file_path[:-1]
    if file_path.endswith("."):
        file_path = file_path[:-1]
    return file_path + ".webp"

In [4]:
q = "SELECT name, mana_cost, cmc, type_line, oracle_text, power, toughness, loyalty FROM cards"
cur.execute(q)
rows = cur.fetchall()

for idx, row in tqdm(enumerate(rows), total=len(rows), desc="Processing cards"):
    new_row = list(row)
    img_path = card_name_to_filename(row[0])
    new_row.append(img_path)
    new_cur.execute("INSERT INTO cards VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", new_row)

    if idx % 1000 == 0:
        conn.commit()

new_conn.commit()


Processing cards: 100%|██████████| 29220/29220 [00:00<00:00, 79406.39it/s]
